In [4]:
import cv2, dlib
import numpy as np
from matplotlib import pyplot as plt
from dlib_align import AlignDlib

In [7]:
class ImgUtils:
    def to_RGB(img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    def to_grayscale(img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    def calc_rect_area(rect):
        return rect.h * rect.w

class FaceUtils:
    def shape_to_np(shape, dtype="int"):
        # initialize the list of (x, y)-coordinates
        coords = np.zeros((68, 2), dtype=dtype)

        # loop over the 68 facial landmarks and convert them
        # to a 2-tuple of (x, y)-coordinates
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)

        # return the list of (x, y)-coordinates
        return coords
    
class Rect:
    def __init__(self, x, y, width, height):
        self.x1 = x
        self.y1 = y
        self.x2 = x + width
        self.y2 = y + height
        self.h = height
        self.w = width
        self.area = ImgUtils.calc_rect_area(self)
        
class AvgFilter:
    def __init__(self, max_window_size, min_window_size=1, confidence_interval=20.0):
        self.max = max_window_size
        self.min = min_window_size
        self.confidence = confidence_interval / 100.0
        
        self.buffer = np.zeros(self.max)
        self.index = 0
    
    def apply_filter(self, rect):
        self.buffer.put(self.index, rect.area)
        avg = np.average(self.buffer)
        
        if self.index == self.max-1: 
            self.index = 0
        else:
            self.index += 1
        
        if avg > (1.0+self.confidence) * rect.area or avg < (1.0-self.confidence) * rect.area:            
            return False
        return True
        
        

In [8]:
#read an image from file
img = cv2.imread("../data/im1.jpg", cv2.IMREAD_COLOR)

classifier = "/opt/opencv-4.0.1/data/haarcascades/haarcascade_frontalface_default.xml"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../models/dlib/shape_predictor_68_face_landmarks.dat")
cascade = cv2.CascadeClassifier(classifier)
aligner = AlignDlib("../models/dlib/shape_predictor_68_face_landmarks.dat")

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    
    if not ret:
        break
    if img is None:
        break
        
    # Our operations on the frame come here
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    rects = cascade.detectMultiScale(gray, 1.5, 1)

    for x,y,h,w in rects:
        rect = dlib.rectangle(x,y,x+w,y+h)
        cv2.rectangle(gray, (x,y),(x+w,y+y),255,3)

        shape = predictor(gray, rect)
        shape = FaceUtils.shape_to_np(shape)
        for (x, y) in shape:
            cv2.circle(gray, (x, y), 4, (0, 0, 255), -1)

        aligned_face = aligner.align(
            imgDim=96,
            rgbImg=ImgUtils.to_RGB(img),
            bb=rect,
            landmarks=shape,
            landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

        cv2.imshow("gray",gray)
        cv2.imshow("align",aligned_face)

        cv2.waitKey(10)
        
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [17]:
cv2.imshow("a",aligned_face)
cv2.waitKey(5000)
cv2.destroyAllWindows()